In [1]:
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import random
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

from env_test import GridEnvironment

In [2]:
# Test GridEnvironment class
N = 10 # 10 EVs

# Get demand data
# TODO get apparent power from real and reactive, use just real for now
with open('Building Load Data/real_power_data.json', 'r') as json_file:
    real_dict = json.load(json_file)
    for key in real_dict.keys(): # get data from first key only (CAPTL_WF)
        demand_data = np.array(real_dict[key])
        break

# Get solar data
with open('PV Generation Data/pv_data.json', 'r') as json_file:
    pv_dict = json.load(json_file)
    for key in pv_dict.keys(): # get data from first key only (CAPTL_WF)
        solar_data = np.array(pv_dict[key])
        break

# Get wind data
with open('Wind Data/wind_data.json', 'r') as json_file:
    wind_dict = json.load(json_file)
    for key in wind_dict.keys(): # get data from first key only (CAPTL_WF)
        wind_data = np.array(wind_dict[key])
        break

grid  = GridEnvironment(N, demand_data, solar_data, wind_data)

In [3]:
solar_data.shape

(427, 96)

In [ ]:
EPISODES = 1000  # Number of episodes to run the training
episode_hours = 24  # Duration of each episode in hours
timestep_length = 0.25  # Duration of each timestep in hours (15 minutes)
episode_length = int(episode_hours / timestep_length)  # Total timesteps per episode
data=0 #TWEAK
env = GridEnvironment(N=10, timestep_length=timestep_length, data=data)  # Initialize your environment with timestep length
state_size = 3 + N  # State includes demand, solar, wind, and N EVs
action_size = 3**N  # Each EV can be in one of three states (-1, 0, 1)
agent = DQNAgent(state_size, action_size)

for e in range(EPISODES):
    day_of_year = 0# Cycling through data day by day
    state = env.reset(day=day_of_year)
    state = np.reshape(state, [1, state_size]) #how do i initialize from data????
    total_reward = 0  # Initialize the total reward for the episode

    for time in range(episode_length):  # Iterate over each timestep within an episode
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -1000  # Negative reward on done (optional)
        total_reward += reward  # Accumulate reward for the episode
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            print(f"Episode: {e+1}/{EPISODES}, Reward: {total_reward}, Steps: {time + 1}")
            break  # Exit the loop when the episode is done

    # Optional: Adjust the learning process based on the performance, e.g., decrease epsilon
    agent.adjust_epsilon()

    # Save model at certain intervals
    if (e + 1) % 100 == 0:
        agent.save_model(f"model_checkpoint_episode_{e+1}.h5")

# Save the final model after all episodes are completed
agent.save_model("final_model.h5")